# Selekcja atrybutów - stepwise

### Algorytm (forward)

1. Zaczynamy od pustego modelu (0 zmiennych wybranych) i przyjmujemy, że wartość kryterium (np. BIC, AIC): C = Inf
2. Szukamy zmiennej, której dodanie do modelu najbardziej zmniejszy kryterium C. 
   * Jeśli wartość C jest mniejsza od poprzedniej wartości - dodajemy zmienną i kontynuujemy
   * Jeśli wartość C dla najlepszego znalezionego w tym kroku modelu nie spadła w stosunku do poprzedniego kroku,
     to nie dołączamy zmiennej i kończymy

## Zadanie 1

Zaimplementuj powyższy algorytm. 

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import sklearn.metrics

# wczytujemy przykladowe dane do testow
# http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/
wine = pd.read_csv("winequality-red.csv", sep = ";")
wine.columns
wine.shape

y = wine.alcohol
y.shape
X = wine.drop("alcohol", axis=1)
X.shape




def penalty_function(y, X_selected, m, method="bic"):
    """
    Mozna rozszerzyc o inne, na razie BIC, AIC
    """
    n, p = X_selected.shape
    y_predict = m.predict(X_selected)
    mse = sklearn.metrics.mean_squared_error(y, y_predict)

    if method == "bic":
        return(n * np.log(mse) + np.log(n)*p )
    elif method == "aic":
        return(n * np.log(mse) + 2*p )
    else:
        raise Exception("improper method")



def stepwise_forward(X, y, max_steps=5, method="bic"):
    """
    Forward feature selection
    X - pd.DataFrame, of shape (n, p)
    y - array-like of shape (n, 1)
    """
    n, p = X.shape
    selected_ind = []

    penalty_best = np.inf

    for step in range(p):

        penalty_best_iter = np.inf
        for i in range(p):
            if i in selected_ind: continue

            # dopasowujemy regresje
            m = LinearRegression()
            m.fit(X.iloc[:, selected_ind + [i]], y)
            penalty_cur = penalty_function(y, X.iloc[:, selected_ind + [i]], m, method)
            print("penalty: {0:9.2f} for {1}".format(penalty_cur, X.columns[i]))

            if penalty_cur < penalty_best_iter:
                penalty_best_iter = penalty_cur
                best_ind = i


        print("\npenalty best: {0} for {1}\n".format(penalty_best_iter, X.columns[best_ind]))
        if penalty_best_iter < penalty_best:
            print("Dołaczamy\n")
            selected_ind.append(best_ind)
            penalty_best = penalty_best_iter
        else:
            print("Nie dołączamy i koniec")
            break

    return selected_ind


In [4]:
result = stepwise_forward(X, y, method="bic")
X_wyb = X.iloc[:, [result[0]]]


penalty:    203.68 for fixed acidity
penalty:    142.97 for volatile acidity
penalty:    190.34 for citric acid
penalty:    206.94 for residual sugar
penalty:    129.60 for chlorides
penalty:    202.05 for free sulfur dioxide
penalty:    140.68 for total sulfur dioxide
penalty:   -242.14 for density
penalty:    140.69 for pH
penalty:    195.71 for sulphates
penalty:   -201.38 for quality

penalty best: -242.13630771473757 for density

Dołaczamy

penalty:   -541.20 for fixed acidity
penalty:   -314.42 for volatile acidity
penalty:   -456.67 for citric acid
penalty:   -354.92 for residual sugar
penalty:   -267.77 for chlorides
penalty:   -248.50 for free sulfur dioxide
penalty:   -297.81 for total sulfur dioxide
penalty:   -237.89 for pH
penalty:   -296.64 for sulphates
penalty:   -606.58 for quality

penalty best: -606.5827447893112 for quality

Dołaczamy

penalty:   -770.33 for fixed acidity
penalty:   -603.32 for volatile acidity
penalty:   -707.17 for citric acid
penalty:   -711.96 f

## Zadanie 2

Przetestuj jego działanie na dobrze znanym zbiorze winequality-red.csv i porównaj wyniki z LASSO i Lasami Losowymi.


## Inne warianty

Powyższy algorytm można zmodyfikować, aby startował z pełneo modelu i w każdym kroku usuwał zmienną, tak aby minimalizować kryterium C (backward)

Można również startować z pustego albo pełneo modelu i w każdym kroku dodawać lub usuwać zmienną, zgodnie z powyższymi zasadami. Koniec nastąpi, gdy ani dodanie, a ni usunięcie zmiennej nie spowoduje spadku wartości kryterium.